In [1]:
ls

 Attention_imdb.ipynb       cnn_kd_twitter.ipynb
 Attention_tiwtter.ipynb    cnn_twitter.ipynb
'bert_gru_tiwtter .ipynb'   kd_atten_lstm_imdb.ipynb
 bert_imdb.ipynb            kd_lstm_imdb.ipynb
 bert_linear_imdb.ipynb     lstm_baseline.ipynb
 bert_twitter.ipynb         lstm_basline_Twitter.ipynb
 cnn_imdb.ipynb             lstm_kd_atten_twitter.ipynb
 cnn_kd.ipynb               lstm_kd_twitter.ipynb


In [2]:
cd ..

/project/6006178/dongxx


In [3]:
cd twitter_baseline

/project/6006178/dongxx/twitter_baseline


In [4]:
import config
config.seed_torch()
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from torchtext.vocab import GloVe,Vocab,Vectors
from tqdm import tqdm
from utils import Twitter_indexing, pad_sequenc,pad_sequencing
from models import CNN_Baseline,LSTMBaseline
from cnn_kd import validate, prepare_dateset,generate_batch,weight_matrix
from bert_train import epoch_time
import torchtext.vocab
import csv
import pandas as pd
import argparse
import logging
import os
import pickle
import sys
import config
config.seed_torch()
from collections import Counter
import time
import copy

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

test = '/home/dongxx/projects/def-parimala/dongxx/data140/test.csv'
valid = '/home/dongxx/projects/def-parimala/dongxx/data140/valid.csv'
train ='/home/dongxx/projects/def-parimala/dongxx/data140/train.csv'

In [6]:
counter2 = Counter({'<unk>': 400002, '<pad>': 400001})
glove = Vectors(name='../glove.6B.100d.txt')
f = open('../glove.6B.{}d.txt'.format(100), 'r')
loop = tqdm(f)
vob = {}
loop.set_description('Load Glove')
for i,line in enumerate(loop):
    values = line.split()
    word = values[0]
    vob[word] =400000-i
counter1 = copy.deepcopy(vob)
f.close()
counter1.update(counter2)
vocab = Vocab(counter1)

Load Glove: : 400000it [00:04, 93568.47it/s]


In [7]:
train_dataset, validation_dataset,test_dataset,labellist = prepare_dateset(train, valid,test,vocab)
testing = DataLoader(test_dataset, collate_fn=generate_batch, batch_size=128, shuffle=False)
cnn_model =CNN_Baseline(vocab_size =400002, nKernel = 64, ksz = [3,4,5],number_class = 2)

cnn_model.embedding_layer.weight.data.copy_(weight_matrix(vocab, glove)).to(device)
cnn_model.embedding_layer.weight.data[1] = torch.zeros(100)
cnn_model.embedding_layer.weight.data[0] = torch.zeros(100)
cnn_model.embedding_layer.weight.requires_grad = False
criterion = nn.CrossEntropyLoss()
criterion.to(device)
cnn_model.to(device)
cnn_model.load_state_dict(torch.load(config.MODEL_CNN_tiwtter_PATH_kd))
start_time = time.time()
test_loss, test_acc,flat_list = validate(testing,cnn_model,criterion,device)
end_time = time.time()
epoch_mins, epoch_secs = epoch_time(start_time, end_time)
print(f' | Epoch Time: {epoch_mins}m {epoch_secs}s')

Start loading training data
Finish loading training data
Start loading validation data
Finish loading validation data
Start loading testing data
Finish loading testing data
prepare training and test sets


0lines [00:00, ?lines/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
1280000lines [16:05, 1326.37lines/s]
160000lines [02:01, 1313.63lines/s]
160000lines [01:58, 1347.42lines/s]


building vocab
 | Epoch Time: 0m 50s


In [8]:
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc * 100:.2f}%')

Test Loss: 0.389 | Test Acc: 83.30%


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt
import itertools

In [ ]:
def plot_confusion_matrix(cm, classes,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """

    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=30)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90, fontsize=22)
    plt.yticks(tick_marks, classes, fontsize=22)

    fmt = '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label', fontsize=25)
    plt.xlabel('Predicted label', fontsize=25)

In [ ]:
pred_list =[item.cpu().numpy() for sublist in flat_list for item in sublist]

In [ ]:
cnf_matrix = confusion_matrix(labellist, pred_list)
plt.figure(figsize=(12,12))
plot_confusion_matrix(cnf_matrix, classes=[0,1], title="Confusion matrix")
plt.show()

In [ ]:
print(classification_report(labellist, pred_list,digits=4))